In [5]:
# !pip install ultralytics

Run this cell as it is

In [1]:
import os
import glob
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping

class_name = { 0: 'Mass',
1: 'Spiculation',
2: 'Suspicious Calcification',
3: 'Architectural Distortion',
4: 'Asymmetry',
5: 'Focal Asymmetry',
6: 'Skin Thickening',
7: 'Global Asymmetry',
8: 'Suspicious Lymph Node',
9: 'Skin Retraction',
10: 'Nipple Retraction'
}

color_dict = {
    0: (255, 0, 0),      # Red
    1: (0, 255, 0),      # Green
    2: (0, 0, 255),      # Blue
    3: (255, 255, 0),    # Yellow
    4: (255, 165, 0),    # Orange
    5: (128, 0, 128),    # Purple
    6: (0, 255, 255),    # Cyan
    7: (255, 192, 203),  # Pink
    8: (128, 128, 0),    # Olive
    9: (0, 0, 0),        # Black
    10: (169, 169, 169)  # Dark Grey
}

E0000 00:00:1737305884.769628  133784 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737305884.773619  133784 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Run this cell as it is

In [2]:
def remove_cache_files(directory):
    cache_files = glob.glob(os.path.join(directory, "*.cache"))
    for cache_file in cache_files:
        os.remove(cache_file)
        print(f"Removed: {cache_file}")
        
with open("/home/rshah133/bcd/dataset.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

Removed: /home/rshah133/breast_cancer_detection/train_val_birads5_test_birads5/val/labels.cache


Model Setup, Change the model and hyperparameters. Keep epochs atleast 50

In [ ]:
model = YOLO('yolov8n.yaml')
results = model.train(data = "/home/rshah133/bcd/dataset.yaml", epochs = 300, imgsz = 640, batch = 6, device = 0, patience = 20)

Run this cell as it is

In [ ]:
with open("/home/rshah133/bcd/test_dataset.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

## Change path and mention the latest train folder generated

* For eg. '/home/rshah133/bcd/runs/detect/train/weights/best.pt'
* I have train as the latest file. Next time I run the code it would change to train2. 
* So please check and change this everytime you run the code

In [10]:
best_model_path = 'runs/detect/train/weights/best.pt'

Predicting on test data

In [ ]:
best_model = YOLO(best_model_path)

test_img_path = data_loaded['val']
results = best_model.predict(source = test_img_path, save = True,  save_txt = True,  conf = 0.1)

Run this cell as it is and take screenshot of the output. This code is for calculating the metrics for test data

In [ ]:
metrics = best_model.val(data = '/home/rshah133/bcd/test_dataset.yaml', conf = 0.15)

## Change the directory of the variable below

* predictions_dir = Directory for the predicted labels. Eg:- "runs/detect/predict/labels" 
* Here predict is variable and can change to predict2, predict3 etc everytime you run it. So change it everytime

In [14]:
predictions_dir = "runs/detect/predict/labels"   

Run this cell as it is

In [ ]:
test_images_dir = data_loaded['val']
test_labels_dir = os.path.join(os.path.dirname(data_loaded['val']), 'labels')


output_labels_dir = "results/labels"  # Directory to save ground truth images
output_pred_dir = "results/predictions"  # Directory to save predicted images

# Create output directory if it doesn't exist
os.makedirs(output_labels_dir, exist_ok=True)
os.makedirs(output_pred_dir, exist_ok=True)

# Function to read YOLO format labels
def read_yolo_labels(label_file):
    labels = []
    with open(label_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts[:5])
            confidence = float(parts[5]) if len(parts) > 5 else None
            labels.append((int(class_id), x_center, y_center, width, height, confidence))
    return labels

# Function to draw bounding boxes on images
def draw_boxes(image, boxes):
    h, w, _ = image.shape
    for box in boxes:
        class_id, x_center, y_center, width, height, confidence = box
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)
        # Draw rectangle
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color_dict[class_id], 2)
    return image

# Loop through each image
for image_file in os.listdir(test_images_dir):
    if image_file.endswith((".jpg", ".png", ".jpeg")):
        base_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(test_images_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading {image_file}")
            continue

        # Read ground truth labels
        ground_truth_file = os.path.join(test_labels_dir, f"{base_name}.txt")
        ground_truth_boxes = read_yolo_labels(ground_truth_file) if os.path.exists(ground_truth_file) else []

        # Read prediction labels
        prediction_file = os.path.join(predictions_dir, f"{base_name}.txt")
        prediction_boxes = read_yolo_labels(prediction_file) if os.path.exists(prediction_file) else []

        # Draw ground truth (green) and predictions (blue)
        image_with_boxes_gt = draw_boxes(image.copy(), ground_truth_boxes)
        image_with_boxes_pt = draw_boxes(image.copy(), prediction_boxes)

        # Save annotated image
        gt_output_path = os.path.join(output_labels_dir, image_file)
        cv2.imwrite(gt_output_path, image_with_boxes_gt)

        pt_output_path = os.path.join(output_pred_dir, image_file)
        cv2.imwrite(pt_output_path, image_with_boxes_pt)

print("Done saving the predictions")

In [ ]:
%matplotlib inline

output_labels_file_names = sorted(os.listdir(output_labels_dir))
output_pred_file_names = sorted(os.listdir(output_pred_dir))

for label_file, pred_file in zip(output_labels_file_names, output_pred_file_names):
    label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
    pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))
    
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Ground Truth')
    axs[0].axis('off')
    axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
    axs[1].set_title('Prediction')
    axs[1].axis('off')
    plt.tight_layout()
    plt.show()